In [149]:
A = matrix( ZZ, [[-8,  0, 10, -1],[-1,  0, 66, -1],[ 1,  0, -1, -3],[-1,  0, -6, -1]])

In [150]:
def pivotChoice(S, A, T, t):
    for i in range(t+1, A.ncols()):
        if(A[i,i] != 0):
            return i, S, A, T
        else:
            for j in range(0, A.nrows()):
                if(A[j,i] != 0):
                    A.swap_rows(i,j)
                    S.swap_rows(i,j)
                    return i, S, A, T
                
    return -1, S, A, T

In [151]:
def pivotImproveRow(S, A, T, i):
    for j in range(0, A.ncols()):
        if(not(A[i,i].divides(A[j,i]))):
            gcd, alpha, beta = xgcd(A[i,i],A[j,i])
            E = matrix.identity(A.nrows())
            E[i,i] = alpha
            E[i,j] = beta
            E[j,i] = -A[j,i]/gcd
            E[j,j] = A[i,i]/gcd
            A = E*A
            S = E*S
            j = 0
    return S, A, T

def pivotImproveCol(S, A, T, i):
    for j in range(0, A.nrows()):
        if(not(A[i,i].divides(A[i,j]))):
            gcd, alpha, beta = xgcd(A[i,i],A[i,j])
            E = matrix.identity(A.ncols())
            E[i,i] = alpha
            E[j,i] = beta
            E[i,j] = -A[i,j]/gcd
            E[j,j] = A[i,i]/gcd
            A = A*E
            T = T*E
            j = 0
    return S, A, T

In [152]:
def eliminateRow(S, A, T, i):
    for j in range(0, A.ncols()):
        if(j != i):
            S.add_multiple_of_row(j,i,-A[j,i]/A[i,i])
            A.add_multiple_of_row(j,i,-A[j,i]/A[i,i])
    return S, A, T

def eliminateCol(S, A, T, i):
    for j in range(0, A.nrows()):
        if(j != i):
            T.add_multiple_of_column(j,i,-A[i,j]/A[i,i])
            A.add_multiple_of_column(j,i,-A[i,j]/A[i,i])
    return S, A, T

In [153]:
def normalize(S, A, T):
    d = A.nonzero_positions()
    for i in range (len(d)):
        if(d[0][0] != 0 or (d[i-1][0]+1 != d[i][0] and i != 0)):
            A.swap_rows(d[i][0],d[i][0]-1)
            S.swap_rows(d[i][0],d[i][0]-1)
            A.swap_columns(d[i][0],d[i][0]-1)
            T.swap_columns(d[i][0],d[i][0]-1) 
            d[i] = [d[i][0]-1,d[i][1]-1]
            i = 0
    
    d = A.nonzero_positions()
    improved = true
    while(improved):
        improved = false
        for i in range(1,len(d)):
            if(not(A[d[i-1]].divides(A[d[i]]))):
                improved = true
                A.add_multiple_of_row(d[i-1][0],d[i][0], 1)
                S.add_multiple_of_row(d[i-1][0],d[i][0], 1)
                S, A, T = pivotImproveCol(S, A, T, d[i-1][0])
                S, A ,T = eliminateRow(S, A, T, d[i-1][0])
                S, A ,T = eliminateCol(S, A, T, d[i-1][0])

    for i in range(0, len(d)):
        if(A[d[i]] < 0):
            A.rescale_row(d[i][0], -1)
            S.rescale_row(d[i][0], -1)
    return S, A ,T

In [154]:
def smithNormalForm(A, B):
    T = matrix.identity(A.base_ring(), A.ncols())
    S = matrix.identity(A.base_ring(), A.nrows())
    pivot = -1
    while(pivot < A.ncols()):
        pivot, S, A, T = pivotChoice(S, A, T, pivot)
        if(pivot == -1):
            break
        while(len(A.nonzero_positions_in_column(pivot)) > 1 or len(A.nonzero_positions_in_row(pivot)) > 1):
            # print("Improve Pivot Row")
            S, A, T = pivotImproveRow(S, A, T, pivot)
            # show(S)
            # show(T)
            # show(A)
            # show(S*B*T)
            # print("Eliminate Row")
            S, A, T = eliminateRow(S, A, T, pivot)
            # show(S)
            # show(T)
            # show(A)
            # show(S*B*T)
            # print("Improve Pivot Col")
            S, A, T = pivotImproveCol(S, A, T, pivot)
            # show(S)
            # show(T)
            # show(A)
            # show(S*B*T)
            # print("Eliminate Col")
            S, A, T = eliminateCol(S, A, T, pivot)
            # show(S)
            # show(T)
            # show(A)
            # show(S*B*T)

    # print("normalize")    
    S, A, T = normalize(S, A, T)
    # show(S)
    # show(B)
    # show(T)
    # show(A)
    # show(S*B*T)
    return S, A, T

In [155]:
# show(A)
# D, U, V = A.smith_form()
# S, B, T = smithNormalForm(copy(A), copy(A))
# show(B)
# show(D)
# show(U*A*V)
# show(S*A*T)

h = 0
for j in range(10):
    k = 0
    for i in range(100):
        A = random_matrix(ZZ,j)
    #show(A)
        D, U, V = A.smith_form()
        S, B, T = smithNormalForm(copy(A), copy(A))
        if(B == D and U*A*V == S*A*T):
            k += 1
        else:
            show(A)
    if(k == 100):
        print("correct for size:", j)
        h += 1
if(h == 100):
    print("correct for all sizes")

correct for size: 0
correct for size: 1
correct for size: 2
correct for size: 3
correct for size: 4
correct for size: 5
correct for size: 6
correct for size: 7
correct for size: 8
correct for size: 9


**Smith Normalform
*1. What's the use of the Smith Normalform
*2. A step by step explanation of the Algorithm over ZZ
    a. Choosing a pivot
    b. Improving the Pivot
    c. Elemination of entrys
    d. Final Step
*3. The Smith normalform over polynomial rings